# OFDM simulation using 64 QAM modulation
OFDM (Orthogonal frequency division multiplexing) is a multicarrier system that is applied in a wide range of wireless transmission systems, such as LTE, WiMAX and DVB-T and DAB. The fundamental concept of a multicarrier system is the division of a high-rate transmitted data stream into several low-rate narrow subcarriers. This way, several advantages are obtained:
- Since the symbol duration is inverse proportional to the symbol rate, each subcarrier has relatively long symbols. Long symbols are robust against multipath fading, as it occurs in wireless systems.
- When a carrier is in a deep fade due to frequency-selectivity of the channel (i.e. the received energy on this carrier is very low), only the data on this subcarrier is lost, instead of the whole stream.
- Multicarrier systems allow easy multi-user resource sharing by allocating different subcarriers to different users.

## Import libraries

In [ ]:
# import cv2
import numpy as np
# import pandas as pd
# import sys
import matplotlib.pyplot as plt
import requests
# import urllib.request
import scipy
# from scipy import interpolate
# from skimage import io
# from google.colab.patches import cv2_imshow
from scipy.ndimage import gaussian_filter1d
import scipy.fftpack
import math as math

## Define Functions

### Helper functions

In [ ]:
def xor(a, b): 
   
    # initialize result 
    result = [] 
   
    # same, then XOR is 0, else 1 
    for i in range(1, len(b)): 
        if a[i] == b[i]: 
            result.append('0') 
        else:
            result.append('1') 
   
    return ''.join(result) 

def covturl(urls):
    resp = requests.get(urls, stream=True).raw
    imgg = np.asarray(bytearray(resp.read()), dtype="uint8")
    return imgg

def mod2div(divident, divisor): 
    # Number of bits to be XORed at a time. 
    pick = len(divisor) 
   
    # Slicing the divident to appropriate 
    # length for particular step 
    tmp = divident[0 : pick] 
   
    while pick < len(divident): 
   
        if tmp[0] == '1': 
   
            # replace the divident by the result 
            # of XOR and pull 1 bit down 
            tmp = xor(divisor, tmp) + divident[pick] 
   
        else:   # If leftmost bit is '0' 
  
            # If the leftmost bit of the dividend (or the 
            # part used in each step) is 0, the step cannot 
            # use the regular divisor; we need to use an 
            # all-0s divisor. 
            tmp = xor('0'*pick, tmp) + divident[pick] 
   
        # increment pick to move further 
        pick += 1
   
    # For the last n bits, we have to carry it out 
    # normally as increased value of pick will cause 
    # Index Out of Bounds. 
    if tmp[0] == '1': 
        tmp = xor(divisor, tmp) 
    else: 
        tmp = xor('0'*pick, tmp) 
   
    checkword = tmp 
    return checkword 

def encodeData(data, key):
    l_key = len(key) 
   
    # Appends n-1 zeroes at end of data 
    appended_data = data + '0'*(l_key-1) 
    remainder = mod2div(appended_data, key) 
   
    # Append remainder in the original data 
    codeword = data + remainder 
    return codeword     

def decodeData(data, key): 
   
    l_key = len(key) 
   
    # Appends n-1 zeroes at end of data 
    appended_data = data + '0'*(l_key-1) 
    remainder = mod2div(appended_data, key) 
   
    return remainder 

def split(word): 
    return [char for char in word]

def PS_FIXED(bits):
    return bits.reshape((-1,))

### Channel functions

In [ ]:
def group_bits(bitc):
    bity = []
    x = 0
    for i in range((len(bitc)//payloadBits_per_OFDM)+1):
        bity.append(bitc[x:x+payloadBits_per_OFDM])
        x = x+payloadBits_per_OFDM
    pp = np.zeros(payloadBits_per_OFDM, dtype = int)
    bity[-1] = pp
    return bity

def SP(bits_serial):
    return bits_serial.reshape((len(dataCarriers), mu))

def PS(bits_parallel):
    return bits_parallel.reshape((-1,))

def Mapping(bits):
    return np.array([mapping_table[tuple(b)] for b in bits])

def Demapping(QAM):
    # array of possible constellation points
    constellation = np.array([x for x in demapping_table.keys()])

    # calculate distance of each RX point to each possible point
    dists = abs(QAM.reshape((-1,1)) - constellation.reshape((1,-1)))

    # for each element in QAM, choose the index in constellation
    # that belongs to the nearest constellation point
    const_index = dists.argmin(axis=1)

    # get back the real constellation point
    hardDecision = constellation[const_index]

    # transform the constellation point into the bit groups
    return np.vstack([demapping_table[C] for C in hardDecision]), hardDecision

def OFDM_symbol(QAM_payload):
    symbol = np.zeros(K, dtype=complex) # the overall K subcarriers
    symbol[pilotCarriers] = pilotValue  # allocate the pilot subcarriers
    symbol[dataCarriers] = QAM_payload  # allocate the pilot subcarriers
    return symbol

def IDFT(OFDM_data):
    return np.fft.ifft(OFDM_data)


def addCP(OFDM_time):
    cp = OFDM_time[-CP:]               # take the last CP samples ...
    return np.hstack([cp, OFDM_time])  # add them to the beginning

def removeCP(signal):
    return signal[CP:(CP+K)]

def channel(signal):
    convolved = np.convolve(signal,channelResponse)
    signal_power = np.mean(abs(convolved**2))
    sigma2 = signal_power * 10**(-SNRdb/10) # calculate noise power based on signal power and SNR
    print ("RX Signal power: %.4f. Noise power: %.4f" % (signal_power, sigma2))
    # Generate complex noise with given variance
    noise = np.sqrt(sigma2/2) * (np.random.randn(*convolved.shape)+1j*np.random.randn(*convolved.shape))
    return convolved + noise

def DFT(OFDM_RX):
    return np.fft.fft(OFDM_RX)

def channelEstimate(OFDM_demod):
    pilots = OFDM_demod[pilotCarriers]  # extract the pilot values from the RX signal
    Hest_at_pilots = pilots / pilotValue # divide by the transmitted pilot values

    # Perform interpolation between the pilot carriers to get an estimate
    # of the channel in the data carriers. Here, we interpolate absolute value and phase
    # separately
    Hest_abs = scipy.interpolate.interp1d(pilotCarriers, abs(Hest_at_pilots), kind='linear')(allCarriers)
    Hest_phase = scipy.interpolate.interp1d(pilotCarriers, np.angle(Hest_at_pilots), kind='linear')(allCarriers)
    Hest = Hest_abs * np.exp(1j*Hest_phase)

    plt.plot(allCarriers, abs(H_exact), label='Correct Channel')
    plt.stem(pilotCarriers, abs(Hest_at_pilots), label='Pilot estimates')
    plt.plot(allCarriers, abs(Hest), label='Estimated channel via interpolation')
    plt.grid(True); plt.xlabel('Carrier index'); plt.ylabel('$|H(f)|$'); plt.legend(fontsize=10)
    plt.ylim(0,2)

    return Hest
def equalize(OFDM_demod, Hest):
    return OFDM_demod / Hest

def get_payload(equalized):
    return equalized[dataCarriers]

def channelEstimate_FIXED(OFDM_demod):
    pilots = OFDM_demod[pilotCarriers]  # extract the pilot values from the RX signal
    Hest_at_pilots = pilots / pilotValue # divide by the transmitted pilot values

    # Perform interpolation between the pilot carriers to get an estimate
    # of the channel in the data carriers. Here, we interpolate absolute value and phase
    # separately
    Hest_abs = scipy.interpolate.interp1d(pilotCarriers, abs(Hest_at_pilots), kind='linear')(allCarriers)
    Hest_phase = scipy.interpolate.interp1d(pilotCarriers, np.angle(Hest_at_pilots), kind='linear')(allCarriers)
    Hest = Hest_abs * np.exp(1j*Hest_phase)
    return Hest

def SNR_return(num_snr):
    def channel_V(signal):
        convolved = np.convolve(signal, channelResponse)
        signal_power = np.mean(abs(convolved**2))
        sigma2 = signal_power * 10**(-SNRdb/10) # calculate noise power based on signal power and SNR
        # Generate complex noise with given variance
        noise = np.sqrt(sigma2/2) * (np.random.randn(*convolved.shape)+1j*np.random.randn(*convolved.shape))
        return convolved + noise
    SNR_Array = np.arange(0,num_snr,0.05)
    bitsnr = bits_groups[10]
    ber = []
    for i in SNR_Array:
        SNRdb = i
        bits_SP = SP(bitsnr)
        QAM = Mapping(bits_SP)
        OFDM_data = OFDM_symbol(QAM)
        OFDM_time = IDFT(OFDM_data)
        OFDM_withCP = addCP(OFDM_time)
        OFDM_Tx = OFDM_withCP
        OFDM_Rx = channel_V(OFDM_withCP)
        OFDM_RX_noCP = OFDM_Rx[(CP):(CP+K)]
        OFDM_demod = DFT(OFDM_RX_noCP)
        Hest = channelEstimate_FIXED(OFDM_demod)
        equalized_Hest = equalize(OFDM_demod, Hest)
        QAM_est = get_payload(equalized_Hest)
        PS_est, hardDecision = Demapping(QAM_est)
        bits_est = PS_FIXED(PS_est)
        ber.append(np.sum(abs(bitsnr-bits_est))/len(bitsnr))
    return SNR_Array, ber

def channel_X(signal):
    convolved = np.convolve(signal,channelResponse)
    signal_power = np.mean(abs(convolved**2))
    sigma2 = signal_power * 10**(-SNRdb/10) # calculate noise power based on signal power and SNR
    # Generate complex noise with given variance
    noise = np.sqrt(sigma2/2) * (np.random.randn(*convolved.shape)+1j*np.random.randn(*convolved.shape))
    return convolved + noise

def pro_bits(SNR_X, bit):
    ber = 0
    bit_rx1 = []
    SNRdb = SNR_X
    for i in bit:
        bits_SP = SP(i)
        QAM = Mapping(bits_SP)
        OFDM_data = OFDM_symbol(QAM)
        OFDM_time = IDFT(OFDM_data)
        OFDM_withCP = addCP(OFDM_time)
        OFDM_Tx = OFDM_withCP
        OFDM_Rx = channel_X(OFDM_withCP)
        OFDM_RX_noCP = OFDM_Rx[(CP):(CP+K)]
        OFDM_demod = DFT(OFDM_RX_noCP)
        Hest = channelEstimate_FIXED(OFDM_demod)
        equalized_Hest = equalize(OFDM_demod, Hest)
        QAM_est = get_payload(equalized_Hest)
        PS_est, hardDecision = Demapping(QAM_est)
        bits_est = PS_FIXED(PS_est)
        ber += np.sum(abs(i-bits_est))/len(i)
        bit_rx1.append(bits_est)
    print("Total BER is :" + str(ber / ((len(img_bits_tx)//payloadBits_per_OFDM)+1)))
    print("All frames are sent")
    return bit_rx1

## Main

### Sender Part

In [ ]:
# url = input("Enter your url :")
# img = covturl(url)
# img = cv2.imdecode(img, cv2.IMREAD_COLOR)
# img = cv2.imread("images/test.bmp", cv2.IMREAD_GRAYSCALE)
# img = cv2.imread("images/medium.bmp", cv2.IMREAD_COLOR)
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

img = plt.imread("images/medium.bmp")

plt.imshow(img)
# plt.savefig("results/input.png")
plt.show()

#### Data Pre-rocessing

First we get all the pixels value of the image and lay them in a 1D array. This array is gonna have a size of `width x height x channel`

In [ ]:
pixels = img.reshape(img.shape[0]*img.shape[1]*img.shape[2]) # width x height x channel
print(pixels)
print(pixels.size)

Next we convert it into binary

In [ ]:
img_bytes = [format(pixel, "08b") for pixel in pixels] # This line below does the exact same thing as 3 lines below 
# img_binary = []
# for pixel in pixels:
#     img_binary.append("{0:b}".format(pixel).zfill(8))
print(img_bytes[0])

Subsequently, we add 1101 at the end of each byte

In [ ]:
# CRC adder with key "1101"
key = "1101"
img_en = [encodeData(i, key) for i in img_bytes] # This line below does the exact same thing as 3 lines below
# img_en = []
# for i in img_binary:
#     img_en.append(encodeData(i, key))
    
print(img_en[0])

In [ ]:
img_bits = [split(i) for i in np.array(img_en)]
# img_bits = []
# for i in np.array(img_en):
#     img_bits.append(split(i))
img_bits = np.vstack(img_bits)
print(img_bits[0])

In [ ]:
# img_ready_for_trans
img_bits_tx = np.array([int(i[j]) for i in img_bits for j in range(11)])
# img_bits_tx = []
# for i in img_bits:
#     for j in range(11):
#         img_bits_tx.append(int(i[j]))
print(img_bits_tx[:11])

### Channel Part

In the following OFDM example, we will go through each block and describe its operation. However, before let us define some parameters that are used for the OFDM system:
- The number of subcarriers $K$ describes, how many subcarriers are available in the OFDM system.
- The length of the cyclic prefix (CP) denotes the number of samples that are copied from the end of the modulated block to the beginning, to yield a cyclic extension of the block.
- The number of pilots $P$ in the OFDM symbol describes, how many carriers are used to transmit known information (i.e. pilots). Pilots will be used at the receiver to estimate the wireless channel between transmitter and receiver. Further, we also define the value that each pilots transmits (which is known to the receiver).

In [ ]:
# CHANNEL SETTING

# CA
K = 128                                     # number of OFDM subcarriers
CP = K // 4                                 # length of the cyclic prefix: 25% of the block
P = 8                                       # number of pilot carriers per OFDM block
SNRdb = 20                                  # signal to noise-ratio in dB at the receiver
pilotValue = 7+7j                           # The known value each pilot transmits
allCarriers = np.arange(K)                  # indices of all subcarriers ([0, 1, ... K-1])
pilotCarriers = allCarriers[::K//P]         # Pilots is every (K/P)th carrier.

# For convenience of channel estimation, the last carrier is also be a pilot
pilotCarriers = np.hstack([pilotCarriers, np.array([allCarriers[-1]])])
P = P+1

# data carriers are all remaining carriers
dataCarriers = np.delete(allCarriers, pilotCarriers)
# print ("allCarriers:   %s" % allCarriers)
# print ("pilotCarriers: %s" % pilotCarriers)
# print ("dataCarriers:  %s" % dataCarriers)

plt.figure(figsize=(15,1))
plt.xlim((-1,K)); plt.ylim((-0.1, 0.3))
plt.xlabel('Carrier index')
plt.yticks([])
plt.grid(True)
plt.plot(pilotCarriers, np.zeros_like(pilotCarriers), 'bo', label='pilot')
plt.plot(dataCarriers, np.zeros_like(dataCarriers), 'ro', label='data')
plt.legend(fontsize=10, ncol=2)
plt.show()

Let's define the modulation index $\mu$ and the corresponding mapping table. We consider 16QAM transmission, i.e. we have $\mu=6$ bits per symbol. Furthermore, the mapping from groups of 6 bits to a 64QAM constellation symbol shall be defined in `mapping_table`.

In [ ]:
mu = 6 # bits per symbol (64QAM)
payloadBits_per_OFDM = len(dataCarriers)*mu # number of payload bits per OFDM symbol

mapping_table = {
    (0,0,0,0,0,0) : -7-7j,
    (0,0,0,0,0,1) : -7-5j,
    (0,0,0,0,1,1) : -7-3j,
    (0,0,0,0,1,0) : -7-1j,
    (0,0,0,1,1,0) : -7+1j,
    (0,0,0,1,1,1) : -7+3j,
    (0,0,0,1,0,1) : -7+5j,
    (0,0,0,1,0,0) : -7+7j,
    (0,0,1,1,0,0) : -5+7j,
    (0,0,1,1,0,1) : -5+5j,
    (0,0,1,1,1,1) : -5+3j,
    (0,0,1,1,1,0) : -5+1j,
    (0,0,1,0,1,0) : -5-1j,
    (0,0,1,0,1,1) : -5-3j,
    (0,0,1,0,0,1) : -5-5j,
    (0,0,1,0,0,0) : -5-7j,
    (0,1,1,0,0,0) : -3-7j,
    (0,1,1,0,0,1) : -3-5j,
    (0,1,1,0,1,1) : -3-3j,
    (0,1,1,0,1,0) : -3-1j,
    (0,1,1,1,1,0) : -3+1j,
    (0,1,1,1,1,1) : -3+3j,
    (0,1,1,1,0,1) : -3+5j,
    (0,1,1,1,0,0) : -3+7j,
    (0,1,0,1,0,0) : -1+7j,
    (0,1,0,1,0,1) : -1+5j,
    (0,1,0,1,1,1) : -1+3j,
    (0,1,0,1,1,0) : -1+1j,
    (0,1,0,0,1,0) : -1-1j,
    (0,1,0,0,1,1) : -1-3j,
    (0,1,0,0,0,1) : -1-5j,
    (0,1,0,0,0,0) : -1-7j,
    (1,1,0,0,0,0) : 1-7j,
    (1,1,0,0,0,1) : 1-5j,
    (1,1,0,0,1,1) : 1-3j,
    (1,1,0,0,1,0) : 1-1j,
    (1,1,0,1,1,0) : 1+1j,
    (1,1,0,1,1,1) : 1+3j,
    (1,1,0,1,0,1) : 1+5j,
    (1,1,0,1,0,0) : 1+7j,
    (1,1,1,1,0,0) : 3+7j,
    (1,1,1,1,0,1) : 3+5j,
    (1,1,1,1,1,1) : 3+3j,
    (1,1,1,1,1,0) : 3+1j,
    (1,1,1,0,1,0) : 3-1j,
    (1,1,1,0,1,1) : 3-3j,
    (1,1,1,0,0,1) : 3-5j,
    (1,1,1,0,0,0) : 3-7j,
    (1,0,1,0,0,0) : 5-7j,
    (1,0,1,0,0,1) : 5-5j,
    (1,0,1,0,1,1) : 5-3j,
    (1,0,1,0,1,0) : 5-1j,
    (1,0,1,1,1,0) : 5+1j,
    (1,0,1,1,1,1) : 5+3j,
    (1,0,1,1,0,1) : 5+5j,
    (1,0,1,1,0,0) : 5+7j,
    (1,0,0,1,0,0) : 7+7j,
    (1,0,0,1,0,1) : 7+5j,
    (1,0,0,1,1,1) : 7+3j,
    (1,0,0,1,1,0) : 7+1j,
    (1,0,0,0,1,0) : 7-1j,
    (1,0,0,0,1,1) : 7-3j,
    (1,0,0,0,0,1) : 7-5j,
    (1,0,0,0,0,0) : 7-7j,
}
for d5 in [0,1]:
    for d4 in [0,1]:
        for d3 in [0,1]:
            for d2 in [0,1]:
                for d1 in [0,1]:
                    for d0 in [0,1]:
                        D = (d5,d4,d3,d2,d1,d0)
                        Q = mapping_table[D]
                        plt.plot(Q.real,Q.imag, 'bo')
                        plt.text(Q.real,Q.imag+0.5, "".join(str(x) for x in D), ha = 'center')

# plt.title("Mapping Table 6 symbol")
plt.grid(True)
plt.xticks(range(-7,8,2))
plt.yticks(range(-7,8,2))
plt.xlim((-8, 8)); plt.ylim((-8,8))
plt.xlabel('Real part (I)')
plt.ylabel('Imaginary part (Q)')
plt.show()
# plt.savefig("results/mapping.png")

We group the bits into a group of `payloadBits_per_OFDM`, which in this case, equals 714

In [ ]:
bits_groups = group_bits(img_bits_tx)
print(bits_groups[0])

In [ ]:
## in this part, we demonstate selecting only 1 frame to send. ##
demapping_table = {v : k for k, v in mapping_table.items()}
channelResponse = np.array([1,0,0.1+0.1j]) # the impulse response of the wireless channel
H_exact = np.fft.fft(channelResponse, K)
plt.plot(allCarriers, abs(H_exact))
plt.show()
# plt.savefig("results/oneframe.png")

The `bits` are now sent to a serial-to-parallel converter, which groups the bits for the OFDM frame into a groups of $\mu$ bits (i.e. one group for each subcarrier):

In [ ]:
# Serial/Parallel
bits = np.array(bits_groups[2]) #we select frame 2 to send
bits_SP = SP(bits) # we select frame 2 to send
print ("First 5 bit groups:")
print (bits_SP[:5,:])

Now, the bits groups are sent to the mapper. The mapper converts the groups into complex-valued constellation symbols according to the `mapping_table`.

In [ ]:
QAM = Mapping(bits_SP)
print ("First 5 QAM symbols:")
print (QAM[:5])

The next step (which is not shown in the diagram) is the allocation of different subcarriers with data and pilots. For each subcarrier we have defined wether it carries data or a pilot by the arrays `dataCarriers` and `pilotCarriers`.

Next, to create the overall OFDM data, we need to put the data and pilots into the OFDM carriers.

Now, the OFDM carriers contained in `OFDM_data` can be transformed to the time-domain by means of the IDFT operation. 

Subsequently, we add a cyclic prefix to the symbol. This operation concatenates a copy of the last `CP` samples of the OFDM time domain signal to the beginning. This way, a cyclic extension is achieved. The CP fulfills two tasks:

1. It isolates different OFDM blocks from each other when the wireless channel contains multiple paths, i.e. is frequency-selective.
2. It turns the linear convolution with the channel into a circular one. Only with a circular convolution, we can use the single-tap equalization OFDM is so famous for.

In [ ]:
OFDM_data = OFDM_symbol(QAM)
OFDM_time = IDFT(OFDM_data)
OFDM_withCP = addCP(OFDM_time)

print("Number of OFDM carriers in frequency domain: ", len(OFDM_data))
print("Number of OFDM samples in time-domain before CP: ", len(OFDM_time))
print("Number of OFDM samples in time domain with CP: ", len(OFDM_withCP))

Now, the signal is sent to the antenna and sent over the air to the receiver. In between both antennas, there is the wireless channel. We model this channel as a static multipath channel with impulse response `channelResponse`. Hence, the signal at the receive antenna is the convolution of the transmit signal with the channel response. Additionally, we add some noise to the signal according to the given SNR value:

In [ ]:
OFDM_TX = OFDM_withCP
OFDM_RX = channel(OFDM_TX)

plt.figure(figsize=(8,2))
plt.plot(abs(OFDM_TX), label='TX signal')
plt.plot(abs(OFDM_RX), label='RX signal')
plt.legend(fontsize=10)
plt.xlabel("Time") 
plt.ylabel("|x(t)|")
plt.grid(True)
plt.show()
# plt.savefig("results/tx_rx.png")

Now, at the receiver the CP is removed from the signal and a window of $K$ samples is extracted from the received signal.

Afterwards, the signal is transformed back to the frequency domain, in order to have the received value on each subcarrier available.

As the next step, the wireless channel needs to be estimated. For illustration purposes, we resort to a simple zero-forcing channel estimation followed by a simple interpolation. The principle of channel estimation is as follows:

The transmit signal contains pilot values at certain pilot carriers. These pilot values and their position in the frequency domain (i.e. the pilot carrier index) are known to the receiver. From the received information at the pilot subcarriers, the receiver can estimate the effect of the wireless channel onto this subcarrier (because it knows what was transmitted and what was received). Hence, the receiver gains information about the wireless channel at the pilot carriers. However, it wants to know what happened at the data carriers. To achieve this, it interpolates the channel values between the pilot carriers to get an estimate of the channel in the data carriers.

Now that the channel is estimated at all carriers, we can use this information in the channel equalizer step. Here, for each subcarrier, the influence of the channel is removed such that we get the clear (only noisy) constellation symbols back.

The next step (not shown in the diagram) is to extract the data carriers from the equalized symbol. Here, we throw away the pilot carriers, as they do not provide any information, but were used for the channel estimation process.

In [ ]:
OFDM_RX_noCP = removeCP(OFDM_RX)
OFDM_demod = DFT(OFDM_RX_noCP)
Hest = channelEstimate(OFDM_demod)
equalized_Hest = equalize(OFDM_demod, Hest)
QAM_est = get_payload(equalized_Hest)
plt.plot(QAM_est.real, QAM_est.imag, 'bo', scalex = True, scaley = True)
plt.show()

Now, that the constellation is obtained back, we need to send the complex values to the demapper, to transform the constellation points to the bit groups. In order to do this, we compare each received constellation point against each possible constellation point and choose the constellation point which is closest to the received point. Then, we return the bit-group that belongs to this point.

In [ ]:
PS_est, hardDecision = Demapping(QAM_est)
for qam, hard in zip(QAM_est, hardDecision):
    plt.plot([qam.real, hard.real], [qam.imag, hard.imag], 'b-o')
    plt.plot(hardDecision.real, hardDecision.imag, 'ro')

In the diagram above, the blue points are the received QAM points, where as the the red points connected to them are the closest possible constellation points, and the bit groups corresponding to these red points are returned. 

Then, the bit groups need to be converted to a serial stream of bits, by means of parallel to serial conversion.

Finally, we can calculate the bit error rate

In [ ]:
bits_est = PS(PS_est)
print ("Obtained Bit error rate: ", np.sum(abs(bits-bits_est))/len(bits))

In [ ]:
SNR_fp, ber_fp = SNR_return(40)
plt.plot(SNR_fp, ber_fp)
plt.xlabel("SNR(E0/N0)")
plt.ylabel("Bit error rate(%)")
plt.grid(True)
# plt.savefig("results/ber.png")
plt.show()

In [ ]:
ysmoothed = gaussian_filter1d(ber_fp, sigma=15)
plt.plot(SNR_fp, ysmoothed, 'c')
plt.xlabel("SNR(E0/N0)")
plt.ylabel("Bit error rate(%)")
plt.grid(True)
plt.show()

In [ ]:
# The expression of wave in time domain & frequency domain
bit_x = 2
a = 1
b = 3

sin_time = []
#import math as math
# Number of sample points
N = 400
# sample spacing
T = 1 / 0.32e12
t = np.linspace(0.0, N*T, N)
fd = 0
fc = 24e9
s = np.zeros(len(t), dtype = float)
bits_SP = SP(bits_groups[bit_x])
QAM = Mapping(bits_SP)
OFDM_data = OFDM_symbol(QAM)
OFDM_time_V = IDFT(OFDM_data)
for i in OFDM_time_V:
    sin_time.append(abs(i)*np.sin(((2*np.pi)*(fc+fd)*t) + math.atan(i.imag/i.real)))
    fd += 1e9
for j in range(len(t)):
    for i in sin_time:
        s[j] += i[j]
plt.plot(t,s)
plt.grid(True)
plt.xlabel("Time(s)")
plt.ylabel("Amplitude(V)")
plt.title("OFDM in time-domain")
plt.show()
#part2
for i in range(a,b):
    plt.plot(t, sin_time[i])
    plt.grid(True)
plt.show()
#part3
for i in range(len(sin_time)):
    yf = scipy.fftpack.fft(sin_time[i])
    xf = np.linspace(0.0, 1.0/(2.0*T), int(N/2))
    plt.plot(xf, 2.0/N * np.abs(yf[:N//2]))
plt.grid(True)
plt.title("OFDM in frequency-domain")
plt.ylabel("|H(f)|")
plt.xlabel("Frequency(Hz)")
plt.show()


#look closely in 2 frequncies spiked
# Number of sample points
sin_time_x = []
G = 75 #N
# sample spacing
M = 1 / 0.0580e12 #T
O = np.linspace(0.0, G*M, G) #t
fd = 0
fc = 24e9
s = np.zeros(len(O), dtype = float)
for i in OFDM_time_V:
    sin_time_x.append(abs(i)*np.sin(((2*np.pi)*(fc+fd)*O) + math.atan(i.imag/i.real)))
    fd += 1e9
for i in range(len(sin_time[0:2])):
    yf = scipy.fftpack.fft(sin_time[i])
    xf = np.linspace(0.0, 1.0/(2.0*M), int(G/2))
    plt.plot(xf, 2.0/G * np.abs(yf[:G//2]))
plt.grid(True)
plt.title("OFDM in frequency-domain")
plt.ylabel("|H(f)|")
plt.xlabel("Frequency(Hz)")
plt.show()

### Receiver Part

In [ ]:
SNR = 1 # Change to adjust channel quaility
bit_rx = pro_bits(SNR, bits_groups)

bit_rx = np.array(bit_rx)
img_fr = []
img_rfr = []
for i in bit_rx:
  img_fr.append(split(i))
img_fr = np.vstack(img_fr)

img_rfr = []
for i in img_fr:
    for j in range(len(img_fr[:2][1])):
        img_rfr.append(int(i[j]))
img_rfr = np.array(img_rfr)

img_rfr1 = []
for i in range(len(img_bits_tx)):
    x = img_rfr[i]
    img_rfr1.append(x)
img_rfr1 = np.array(img_rfr1)

img_rea = []
k = 0
for i in range(len(img_rfr1)):
    img_rea.append(img_rfr1[k:k+11])
    k+=11
    
img_done = []
for i in img_rea[:]:
    x = []
    for j in range(len(i)):
        x.append(str(i[j]))
    img_done.append(x)

sep = ''
img_fin = []
for i in img_done:
    img_fin.append(sep.join(i))

In [ ]:
#remove CRC and compare
img_crc = [decodeData(i, key) for i in img_fin]
# img_crc = []
# for i in img_fin:
#     img_crc.append(decodeData(i, key))

count = 0
for i in img_crc:
    if i != '000':
        if i!= '00':
            count += 1

print("Amount of total bits: %10d" %(len(img_bits_tx)))
print("Amount of error bit(s): %4d" %count)

In [ ]:
#decode
img_dec = []
for i in img_fin:
    img_dec.append(i[0:8])
img_dec = np.array(img_dec)


In [ ]:
##We can change SNR to see another quality of the image after passing the channel 
img_back = []
for i in range(len(img_bytes)):
    img_back.append(int(img_dec[i],2))
img_back = np.array(img_back)
img_back = img_back.reshape(img.shape[0], img.shape[1], img.shape[2])
plt.imshow(img_back)
plt.show()